In [114]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType
from pyspark.sql.functions import monotonically_increasing_id, format_number
from pyspark.ml.feature import VectorAssembler, StringIndexer
from pyspark.ml.regression import GeneralizedLinearRegression
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.regression import DecisionTreeRegressor
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.regression import IsotonicRegression
from pyspark.ml.regression import LinearRegression

from pyspark.ml.regression import GBTRegressor
from pyspark.ml.regression import FMRegressor
from pyspark.sql import SparkSession, Row
from pyspark.sql.functions import col
from pyspark.ml import Pipeline

In [115]:
# Создание SparkSession
spark = SparkSession.builder.appName("CrabAgePrediction").getOrCreate()
# Загружаем датасет
data = spark.read.csv("CrabAgePrediction.csv", header = True, inferSchema = True)

data.printSchema()
data.show()

root
 |-- Sex: string (nullable = true)
 |-- Length: double (nullable = true)
 |-- Diameter: double (nullable = true)
 |-- Height: double (nullable = true)
 |-- Weight: double (nullable = true)
 |-- Shucked Weight: double (nullable = true)
 |-- Viscera Weight: double (nullable = true)
 |-- Shell Weight: double (nullable = true)
 |-- Age: integer (nullable = true)

+---+------+--------+------+-----------+--------------+--------------+------------+---+
|Sex|Length|Diameter|Height|     Weight|Shucked Weight|Viscera Weight|Shell Weight|Age|
+---+------+--------+------+-----------+--------------+--------------+------------+---+
|  F|1.4375|   1.175|0.4125| 24.6357155|    12.3320325|     5.5848515|    6.747181|  9|
|  M|0.8875|    0.65|0.2125| 5.40057975|     2.2963095|    1.37495075|   1.5592225|  6|
|  I|1.0375|   0.775|  0.25| 7.95203475|      3.231843|    1.60174675|  2.76407625|  6|
|  F| 1.175|  0.8875|  0.25|13.48018725|    4.74854125|    2.28213475|   5.2446575| 10|
|  I|0.8875|  0.6

In [116]:
# Преобразуем Пол краба в числа
indexer = StringIndexer(inputCol="Sex", outputCol="SexIndex")
# Преобразовываем данные. Начинаем с выбора признаков
features = ["SexIndex", "Length", "Diameter" , "Height", "Weight", "Shucked Weight", "Viscera Weight", "Shell Weight"]
# Создаем вектор этих признаков
assembler = VectorAssembler(inputCols=features, outputCol="features")
# Разделяем данные на обучение и тест
(train, test) = data.randomSplit([0.7, 0.3])
# Создаем кортеж среднеквадратичных ошибок
tupleRMSE = ()
Pred_arr = []

train.show(3)
test.show(3)

print(train.count())
print(test.count())

+---+------+--------+------+----------+--------------+--------------+------------+---+
|Sex|Length|Diameter|Height|    Weight|Shucked Weight|Viscera Weight|Shell Weight|Age|
+---+------+--------+------+----------+--------------+--------------+------------+---+
|  F|0.6875|  0.4875| 0.175|   2.26796|     0.8788345|    0.60951425|   0.7087375|  5|
|  F| 0.725|  0.5625|0.1875|   3.96893|    1.45999925|    0.66621325|     1.13398|  5|
|  F|0.7625|  0.5625| 0.175|4.20990075|    1.65844575|    0.94970825|   1.2757275|  7|
+---+------+--------+------+----------+--------------+--------------+------------+---+
only showing top 3 rows

+---+------+--------+------+---------+--------------+--------------+------------+---+
|Sex|Length|Diameter|Height|   Weight|Shucked Weight|Viscera Weight|Shell Weight|Age|
+---+------+--------+------+---------+--------------+--------------+------------+---+
|  F| 0.725|   0.525|0.1875|7.7961125|     3.2034935|    1.91359125|   0.9922325|  6|
|  F| 0.825|    0.65| 

In [117]:
# Создаем модель
rf = RandomForestRegressor(featuresCol = "features", labelCol= "Age")
# Создаем конвейер
pipelineRF = Pipeline(stages = [indexer, assembler, rf])
# Обучение модели
modelRF = pipelineRF.fit(train)
# Предсказание возраста на тестовой выборке
predictionsRF = modelRF.transform(test).withColumnRenamed("prediction", "predictionsRF")
Pred_arr.append(predictionsRF)

In [118]:
# Оценка модели
evaluator = RegressionEvaluator(labelCol = "Age", predictionCol = "predictionsRF", metricName = "rmse")
rmseRF = evaluator.evaluate(predictionsRF)
tupleRMSE += (rmseRF,)
print("Root Mean Squared Error (RMSE): %s" % rmseRF)
# Вывод результата и выключение SparkSession
predictionsRF.select("Age", "predictionsRF").show()

Root Mean Squared Error (RMSE): 2.260448757104348
+---+-----------------+
|Age|    predictionsRF|
+---+-----------------+
|  6|6.811880140021161|
|  9|8.013312388754805|
|  6|8.300477498340209|
| 10|7.946928207963846|
|  5| 8.29030086143862|
|  8|7.981658388263095|
| 10|8.943553586740432|
|  9| 9.00774383674751|
| 10| 8.88381159718502|
|  7|9.229501979302102|
| 10|9.003995771274493|
| 13|8.637029566204749|
|  9|8.813906985276649|
| 10| 9.52206116732336|
|  9|9.180597184275983|
| 10|9.456747540091062|
| 11|9.180597184275983|
|  7|9.911593737310705|
|  8| 9.44908397982967|
| 13|9.361210472288825|
+---+-----------------+
only showing top 20 rows



In [119]:
# Создание модели градиентного бустинга
gbt = GBTRegressor(featuresCol="features", labelCol= "Age")
# Создание конвейера
pipelineGBT = Pipeline(stages = [indexer, assembler, gbt])
# Обучение модели
modelGBT = pipelineGBT.fit(train)
# Предсказание возраста на тестовой выборке
predictionsGBT = modelGBT.transform(test).withColumnRenamed("prediction", "predictionsGBT")
Pred_arr.append(predictionsGBT)

In [120]:
# Оценка модели
evaluator = RegressionEvaluator(labelCol = "Age", predictionCol = "predictionsGBT", metricName = "rmse")
rmseGBT = evaluator.evaluate(predictionsGBT)
tupleRMSE += (rmseGBT,)
print("Root Mean Squared Error (RMSE): %s" % rmseGBT)
# Вывод результата и выключение SparkSession
predictionsGBT.select("Age", "predictionsGBT").show()
#spark.stop()

Root Mean Squared Error (RMSE): 2.2774418187359338
+---+------------------+
|Age|    predictionsGBT|
+---+------------------+
|  6| 6.966501840694569|
|  9| 8.606467330660061|
|  6|  8.36087730590337|
| 10| 8.606467330660061|
|  5| 8.308248237680596|
|  8| 7.902399276592007|
| 10| 8.653354613524797|
|  9| 8.653354613524797|
| 10| 8.653354613524797|
|  7| 8.410614926765058|
| 10| 8.653354613524797|
| 13| 8.519906136223105|
|  9|  9.55306996781089|
| 10|   9.8838684013514|
|  9|  9.55306996781089|
| 10| 9.532856935364114|
| 11|  9.55306996781089|
|  7|11.175572128265193|
|  8| 9.770033721463044|
| 13| 9.770033721463044|
+---+------------------+
only showing top 20 rows



In [121]:
# Создание модели линейной регрессии
lr = LinearRegression(featuresCol="features", labelCol="Age", maxIter=10, regParam=0.3, elasticNetParam=0.8) 
# Создание конвейера
pipelineLR = Pipeline(stages = [indexer, assembler, lr])
# Fit the model
modelLR = pipelineLR.fit(train)
# Предсказание возраста на тестовой выборке
predictionsLR = modelLR.transform(test).withColumnRenamed("prediction", "predictionsLR")
Pred_arr.append(predictionsLR)

In [122]:
# Оценка модели
evaluator = RegressionEvaluator(labelCol = "Age", predictionCol = "predictionsLR", metricName = "rmse")
rmseLR = evaluator.evaluate(predictionsLR)
tupleRMSE += (rmseLR,)
print("Root Mean Squared Error (RMSE): %s" % rmseLR)
# Вывод результата и выключение SparkSession
predictionsLR.select("Age", "predictionsLR").show()
#spark.stop()

Root Mean Squared Error (RMSE): 2.490650985826046
+---+-----------------+
|Age|    predictionsLR|
+---+-----------------+
|  6|7.171451480599124|
|  9|7.593180043486409|
|  6|7.527697575711218|
| 10|7.450802088033083|
|  5|7.527697575711218|
|  8|7.593180043486409|
| 10|7.931402989307753|
|  9|7.974837630178287|
| 10|8.123223302061863|
|  7|8.236165088321496|
| 10|7.990877740652695|
| 13| 7.95945853264266|
|  9|8.056360208427886|
| 10|8.365146985055537|
|  9|8.353131552195038|
| 10|8.822319433566765|
| 11|8.380526082591164|
|  7|8.905825050632705|
|  8|8.881794184911705|
| 13| 8.72139308016763|
+---+-----------------+
only showing top 20 rows



In [123]:
# Создание модели решающих деервьев
dt = DecisionTreeRegressor(featuresCol="features", labelCol="Age")
# Создание конвейера
pipelineDT = Pipeline(stages = [indexer, assembler, dt])
# Fit the model
modelDT = pipelineDT.fit(train)
# Предсказание возраста на тестовой выборке
predictionsDT = modelDT.transform(test).withColumnRenamed("prediction", "predictionsDT")
Pred_arr.append(predictionsDT)

In [124]:
# Оценка модели
evaluator = RegressionEvaluator(labelCol = "Age", predictionCol = "predictionsDT", metricName = "rmse")
rmseDT = evaluator.evaluate(predictionsDT)
tupleRMSE += (rmseDT,)
print("Root Mean Squared Error (RMSE): %s" % rmseDT)
# Вывод результата и выключение SparkSession
predictionsDT.select("Age", "predictionsDT").show()

Root Mean Squared Error (RMSE): 2.3452812067160207
+---+------------------+
|Age|     predictionsDT|
+---+------------------+
|  6|               7.5|
|  9| 8.435897435897436|
|  6| 8.435897435897436|
| 10| 8.435897435897436|
|  5| 8.435897435897436|
|  8| 8.435897435897436|
| 10| 8.435897435897436|
|  9| 8.435897435897436|
| 10| 8.435897435897436|
|  7| 8.435897435897436|
| 10| 8.435897435897436|
| 13| 8.435897435897436|
|  9| 9.654135338345865|
| 10| 9.654135338345865|
|  9| 9.654135338345865|
| 10| 9.654135338345865|
| 11| 9.654135338345865|
|  7|11.155339805825243|
|  8| 9.654135338345865|
| 13| 9.654135338345865|
+---+------------------+
only showing top 20 rows



In [125]:
# Создание модели изотонической регрессии
ir = IsotonicRegression(featuresCol="features", labelCol="Age")
# Создание конвейера
pipelineIR = Pipeline(stages = [indexer, assembler, ir])
# Fit the model
modelIR = pipelineIR.fit(train)
# Предсказание возраста на тестовой выборке
predictionsIR = modelIR.transform(test).withColumnRenamed("prediction", "predictionsIR")
Pred_arr.append(predictionsIR)

In [126]:
# Оценка модели
evaluator = RegressionEvaluator(labelCol = "Age", predictionCol = "predictionsIR", metricName = "rmse")
rmseIR = evaluator.evaluate(predictionsIR)
tupleRMSE += (rmseIR,)
print("Root Mean Squared Error (RMSE): %s" % rmseIR)
# Вывод результата и выключение SparkSession
predictionsIR.select("Age", "predictionsIR").show()

Root Mean Squared Error (RMSE): 3.2835061133005885
+---+-------------+
|Age|predictionsIR|
+---+-------------+
|  6|         13.0|
|  9|         13.0|
|  6|         13.0|
| 10|         13.0|
|  5|         13.0|
|  8|         13.0|
| 10|         13.0|
|  9|         13.0|
| 10|         13.0|
|  7|         13.0|
| 10|         13.0|
| 13|         13.0|
|  9|         13.0|
| 10|         13.0|
|  9|         13.0|
| 10|         13.0|
| 11|         13.0|
|  7|         13.0|
|  8|         13.0|
| 13|         13.0|
+---+-------------+
only showing top 20 rows



In [127]:
# Создание модели FMRegressor
fm = FMRegressor(featuresCol="features", labelCol="Age", predictionCol="prediction",stepSize=0.001)
# Создание конвейера
pipelineFM = Pipeline(stages = [indexer, assembler, fm])
# Fit the model
modelFM = pipelineFM.fit(train)
# Предсказание возраста на тестовой выборке
predictionsFM = modelFM.transform(test).withColumnRenamed("prediction", "predictionsFM")
Pred_arr.append(predictionsFM)

In [128]:
# Оценка модели
evaluator = RegressionEvaluator(labelCol = "Age", predictionCol = "predictionsFM", metricName = "rmse")
rmseFM = evaluator.evaluate(predictionsFM)
tupleRMSE += (rmseFM,)
print("Root Mean Squared Error (RMSE): %s" % rmseFM)
# Вывод результата и выключение SparkSession
predictionsFM.select("Age", "predictionsFM").show()

Root Mean Squared Error (RMSE): 5.538404021216881
+---+------------------+
|Age|     predictionsFM|
+---+------------------+
|  6|1.6624477013093848|
|  9|1.4528687073400488|
|  6|1.4884681491695524|
| 10| 1.497572709908563|
|  5|1.4197094567093806|
|  8|1.6862643627625282|
| 10|1.6895900994501776|
|  9|1.9282843958762583|
| 10|1.9362676259012805|
|  7| 2.146352898954901|
| 10| 1.841698977993571|
| 13|1.9370921279467912|
|  9| 2.482834469594187|
| 10| 2.637260523210491|
|  9|  2.49116043790531|
| 10|3.3473054131780646|
| 11| 2.578078908266286|
|  7| 3.220765301461028|
|  8| 3.213839523664849|
| 13|3.1819115929492123|
+---+------------------+
only showing top 20 rows



In [129]:
# Модель  `AFTSurvivalRegression`  не подходит для датасета  `CrabAgePrediction`, так как нет информации о том,  
# живы ли были крабы, когда их поймали.

# Эта модель предназначена для анализа данных выживания, где есть данные о времени до события (в данном случае, смерть) 
# и информация о том,  было ли это событие наблюдено (краб умер) или  **цензурировано**  (краб был жив, когда его поймали).  

In [130]:
glr = GeneralizedLinearRegression(featuresCol="features", labelCol="Age", family="gaussian", link="identity", maxIter=10, regParam=0.3)
# Создание конвейера
pipelineGLR = Pipeline(stages = [indexer, assembler, glr])
# Fit the model
modelGLR = pipelineGLR.fit(train)
# Предсказание возраста на тестовой выборке
predictionsGLR = modelGLR.transform(test).withColumnRenamed("prediction", "predictionsGLR")
Pred_arr.append(predictionsGLR)

In [131]:
# Оценка модели
evaluator = RegressionEvaluator(labelCol = "Age", predictionCol = "predictionsGLR", metricName = "rmse")
rmseGLR = evaluator.evaluate(predictionsGLR)
tupleRMSE += (rmseGLR,)
print("Root Mean Squared Error (RMSE): %s" % rmseGLR)
# Вывод результата и выключение SparkSession
predictionsGLR.select("Age", "predictionsGLR").show()

Root Mean Squared Error (RMSE): 2.296742684862444
+---+------------------+
|Age|    predictionsGLR|
+---+------------------+
|  6| 6.157138650384186|
|  9| 7.340004885967327|
|  6| 7.144748345630807|
| 10| 7.250389680451425|
|  5| 7.288170709894404|
|  8| 7.353534224940114|
| 10| 7.802288674287633|
|  9| 7.778505287716175|
| 10| 8.110539491757343|
|  7| 8.092268048478232|
| 10| 7.939679886567855|
| 13|7.9316400578915145|
|  9| 7.890496103148736|
| 10|  8.32628922608177|
|  9| 8.438996086913928|
| 10|  8.77063124874799|
| 11| 8.727006031693433|
|  7| 9.080677824716698|
|  8| 8.958039062702635|
| 13|  8.88751593190273|
+---+------------------+
only showing top 20 rows



In [132]:
# Создаем списки моделей, выполняющихся том же в порядке выше
modelsList = ["RandomForestRegressor", "GBTRegressor", "LinearRegression", "DecisionTreeRegressor", 
              "IsotonicRegression", "FMRegressor", "GeneralizedLinearRegression"]
shortModelsList = ["predictionsRF", "predictionsGBT", "predictionsLR", "predictionsDT", "predictionsIR", 
                   "predictionsFM", "predictionsGLR"]

# Схема нового DataFrame значений RMSE каждой модели
schemaRMSE = StructType([
    StructField(modelsList[0], DoubleType(), True),
    StructField(modelsList[1], DoubleType(), True),
    StructField(modelsList[2], DoubleType(), True),
    StructField(modelsList[3], DoubleType(), True),
    StructField(modelsList[4], DoubleType(), True),
    StructField(modelsList[5], DoubleType(), True),
    StructField(modelsList[6], DoubleType(), True)
])
# Создание DataFrame
rmseDF = spark.createDataFrame([tupleRMSE], schemaRMSE)

In [133]:
for i, df in enumerate(Pred_arr):
    df = df.withColumn("id", monotonically_increasing_id())
    Pred_arr[i] = df

ListSelectedDF = [df.select("Age", "id", shortModelsList[i]) for i, df in enumerate(Pred_arr)]
predDF = ListSelectedDF[0]

for i in range(1, len(ListSelectedDF)):
    predDF = predDF.join(ListSelectedDF[i], on = ["Age", "id"], how = "inner")

predDF = predDF.drop("id")

In [134]:
# Вывод лучшего результата
print(f"Лучшая модель: {modelsList[rmseDF.head().index(min(rmseDF.head()))]} (RMSE: {min(rmseDF.head())})")
# Запись результатов в json-файл
rmseDF.write.json("RMSE.json", mode = 'overwrite')
predDF.write.json("Pred.json", mode = 'overwrite')

Лучшая модель: RandomForestRegressor (RMSE: 2.260448757104348)


In [135]:
spark.stop()